# Original ResNet
- More layers purely leeds to gradient vanishing problems
- It can be solved by normalizing initializations
- As well as add intermediate normalization layers
- Deeper network can also lead to unable to search for optimal as easy as shallow network. **Degradation**
- Residual neural network solved degradation Problem.
- The solver of SGD based on  BP might have difficulty solving many layers of non-linear functions.
- A network might be redundantly deep, resnet make it possible to abandon some depth from time to time.

## Methodology:
- Using CIFRA-10 and ImageNet for benchmarking
- ImageNet case: Use 152 layers.
- This has better performance on other tasks like COCO detection and COCO segmentation.
- It is generic; applicable to vision and non-vision problems.
- 在训练，尝试的过程中：Batch Normalization 起到了关键的作用。




In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from tensorflow.keras import layers

TensorFlow 2.x selected.


## Preparing TPU

In [10]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Initializing the TPU system: 10.104.76.250:8470


INFO:tensorflow:Initializing the TPU system: 10.104.76.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


## Build Flexiable architecture of ResNet
- Data set should be CiFar-10
- Define the residual block

In [0]:

class ResBlock(layers.Layer):
  def __init__(self,name='resblock',channels=64,**kwargs):
    super(ResBlock,self).__init__(name=name,**kwargs)
    self.conv_1 = layers.Conv2D(channels,3,activation='relu'  ,padding='same')
    self.batch_1= layers.BatchNormalization()
    self.conv_2 = layers.Conv2D(channels,3,activation='relu',padding='same')
    self.batch_2= layers.BatchNormalization()

  def call(self,inputs):
    x = self.conv_1(inputs[0])
    x = self.batch_1(x)
    x = self.conv_2(x)
    x = self.batch_2(x)
    index=0
    if len(inputs) > 1:
      index=1
    x = layers.add([inputs[index],x])
    x = tf.nn.relu(x)
    return x

## Construct the network based on ResBlock defined

In [0]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
  period_1 = [ResBlock(channels=64,name='resbock'+str(i)+'period_1') for i in range(3)]
  period_2 = [ResBlock(channels=128,name='resbock'+str(i)+'period_2') for i in range(2)]
  period_3 = [ResBlock(channels=256,name='resbock'+str(i)+'period_3') for i in range(5)]
  period_4 = [ResBlock(channels=512,name='resbock'+str(i)+'period_4') for i in range(5)]

  inputs = keras.Input(shape = (32,32,3),name='resnet')
  x = layers.Conv2D(64,7,activation='relu',padding='same')(inputs)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(2)(x)
  for layer in period_1:
    x = layer([x])

  x = layers.MaxPooling2D(2)(x)
  res_x = layers.Conv2D(128,1)(x) # Pure depth conv
  x = ResBlock(channels=128,name='intermediate_1')(inputs=[x,res_x])
  for layer in period_2:
    x = layer([x])

  x = layers.MaxPooling2D(2)(x)
  res_x = layers.Conv2D(256,1)(x) # Pure depth conv
  x = ResBlock(channels=256,name='intermediate_2')([x,res_x])
  for layer in period_3:
    x = layer([x])

  #x = layers.MaxPooling2D(2)(x)
  res_x = layers.Conv2D(512,1)(x) # Pure depth conv
  x = ResBlock(channels=512,name='intermediate_3')([x,res_x])
  for layer in period_4:
    x = layer([x])

  x = layers.GlobalAveragePooling2D()(x)
  outputs = layers.Dense(10,activation='sigmoid')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  keras.utils.plot_model(model,'model.jpg',show_shapes=True,expand_nested=True)
  model.summary()
  model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['acc'])

## Load CIFRA-10 Dataset
- Preprocess and compress it into 32 * 32
- Using tensorflow dataset

In [0]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
model.fit(x_train, y_train,
          batch_size=1000,
          epochs=1000,
          validation_split=0.2)